In [26]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import random
import numpy as np
import math
import warnings
import missingno as msno
from pprint import pprint
from chinese_calendar import is_workday, is_holiday
from pathlib import Path
from sklearn.preprocessing import PowerTransformer
import scipy.stats as st
from collections import defaultdict
import json

# 固定随机种子，稳定模型效果
random.seed(2021) 
np.random.seed(2021)
%matplotlib inline
warnings.filterwarnings("ignore")
plt.rc("font",family="SimHei",size="10")
plt.rcParams['axes.unicode_minus']=False
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',200)
pd.set_option('expand_frame_repr', False)

# 数据集路径
root = Path(os.getcwd()).resolve().parent / "data"
root2 = Path(os.getcwd()).resolve().parent / "pic"
train = root / "Q3_train.csv"
data_df = pd.read_csv(train)

# index2name存放的是所有特征的名称
index2name = ["carid", "pushDate", "pushPrice", "updatePriceTimeJson", "pullDate", "withdrawDate", "展销时间",
              "品牌 id", "车系 id", "车型 id", "里程", "车辆颜色", "车辆所在城市 id", "国标码",
              "过户次数", "载客人数", "注册日期", "上牌日期", "国别", "厂商类型", "年款", "排量", "变速箱", "燃油类型",
              "新车价"]

# category存放的是类别特征的名称
category = ["品牌 id", "车系 id", "车型 id", "车辆颜色", "车辆所在城市 id", "厂商类型", "燃油类型"] # 类别类型特征

# numerical存放的是数值特征的名称
numerical = ["Feature_5", "年款", "排量", "变速箱", "Feature_2", "Feature_13", "载客人数", "新车价", "target", "fday", "pushPrice"] #  "里程", "厂商类型"实数类型特征, "Feature_12", "Feature_8"

# cross_num存放的是要进行特征交叉的特征的名称
cross_num = ["新车价", "Feature_2", "Feature_12_1"]

# 匿名特征为Feature_i，此处将15个匿名特征加入到index2name中
for _ in range(1, 16):
    index2name.append("Feature_"+str(_))
index2name.append("target")
index2name.append("fday")
index2name.append("sday")
index2name.append("成交周期")


sFea = {}
cx = data_df.groupby("车系 id")
for a, b in cx:
    sFea[a] = list(set(b["车型 id"]))

def metric(y_, y):
    ape = np.abs(y_ - y)
    mape = np.mean(ape)
    return mape

data_df = data_df[data_df["pullDate"] == data_df["withdrawDate"]]

data_df["updatePriceTimeJson"] = data_df[["pushDate", "pushPrice", "updatePriceTimeJson"]].apply(lambda x: {str(x["pushDate"]): x["pushPrice"]} if len(x["updatePriceTimeJson"]) == 2 else dict({str(x["pushDate"]): x["pushPrice"]}, **json.loads(x["updatePriceTimeJson"])), axis=1)

data_df = data_df[data_df["target"] <= 10000]
data_df.sort_values(by="展销时间", inplace=True)


data_df["展销时间"] = pd.to_datetime(data_df["展销时间"])
data_df["注册日期"] = pd.to_datetime(data_df["注册日期"])
data_df["上牌日期"] = pd.to_datetime(data_df["上牌日期"])
data_df["pushDate"] = pd.to_datetime(data_df["pushDate"])
data_df["pullDate"] = pd.to_datetime(data_df["pullDate"])

data_df["fday"] = data_df[["pushDate", "updatePriceTimeJson"]].apply(lambda x: (pd.to_datetime(list(x["updatePriceTimeJson"].keys())[-1]) - x["pushDate"]).days, axis=1)
data_df["sday"] = data_df["成交周期"]  - data_df["fday"]

data_df["成交周期2"] = data_df["成交周期"]
del data_df["成交周期"]
data_df["成交周期"] = data_df["成交周期2"]
del data_df["成交周期2"]

te = pd.read_csv(root / "Q3_processed.csv")
te["展销时间"] = pd.to_datetime(te["展销时间"])
te["注册日期"] = pd.to_datetime(te["注册日期"])
te["上牌日期"] = pd.to_datetime(te["上牌日期"])
te["pushDate"] = pd.to_datetime(te["pushDate"])
te["updatePriceTimeJson"] = te[["pushDate", "pushPrice", "updatePriceTimeJson"]].apply(lambda x: {str(x["pushDate"]): x["pushPrice"]} if len(x["updatePriceTimeJson"]) == 2 else dict({str(x["pushDate"]): x["pushPrice"]}, **json.loads(x["updatePriceTimeJson"])), axis=1)

# add the null
# tr = pd.concat([tr, data_df_null], axis=0)
tr = data_df
tr = tr[((tr["新车价"]-tr["target"]) / tr["新车价"]) < 0.969]
tr = tr[tr["新车价"] > tr["target"]]

# 使用众数填补缺省值
for _ in tr:
    tr[_] = tr[_].fillna(tr[_].value_counts().index[0])

import scipy.stats as st
y = tr["sday"]
plt.figure(figsize=[8,5])
plt.subplot(1,3,1)
plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.subplot(1,3,2)
plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.subplot(1,3,3)
plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)
plt.savefig(root2 / "Q3_0.jpg")

pt_tr = PowerTransformer(method="yeo-johnson")

tr["sday"] = pt_tr.fit_transform(np.array(tr['sday']).reshape(-1,1))
# tr["成交周期"] = np.log(tr["成交周期"])
plt.figure(figsize=[20, 10])
sns.displot(tr['sday'])
plt.savefig(root2 / "Q3_1.jpg")

# ******************构造新特征********************
class FeatureEngine:
    def __init__(self):
        self.ID = defaultdict(lambda: defaultdict(int))
        self.sFea = {}
        self.target_cxx = defaultdict(lambda: defaultdict(int))
        
    @staticmethod
    def add2list(pos, name, stage="train"):
        if stage == "train":
            index2name.insert(pos, name)
            numerical.append(name)
    
    @staticmethod
    def add2list_cat(pos, name, stage="train"):
        if stage == "train":
            index2name.insert(pos, name)
            category.append(name)
    
    # strongFea没啥用，忽略
    def strongFea(self, data_df, stage):
        if stage == "train":
            cx = tr.groupby("车系 id")
            for a, b in cx:
                self.sFea[a] = list(set(b["车型 id"]))
    
            for cx_idx in self.sFea:
                for cxing_idx in self.sFea[cx_idx]:
                    if len(data_df[(data_df["车系 id"]==cx_idx) & (data_df["车型 id"]==cxing_idx)]) <= 5:
                        self.target_cxx[cx_idx][cxing_idx] = 0
                    else:
                        self.target_cxx[cx_idx][cxing_idx] = float(data_df[(data_df["车系 id"]==cx_idx) & (data_df["车型 id"]==cxing_idx)]["target"].mean())
            
        data_df.insert(len(data_df.columns)-1, "车系车型_mean", data_df.apply(lambda x: self.target_cxx[x["车系 id"]][x["车型 id"]], axis=1))
        data_df["车系车型_mean"] = data_df["车系车型_mean"].fillna(data_df["车系车型_mean"].mean())
        self.add2list(len(index2name)-1, "车系车型_mean", stage)
    
    # 下面一系列的get函数是类似的，按照某个特征进行聚类，然后求mean，count，max......
    def getMean(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_mean"][list(set(v[fea_name]))[0]] = v["target"].mean()
            data_df.insert(len(data_df.columns)-1, fea_name+"_mean", data_df[fea_name].map(self.ID[fea_name+"_mean"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_mean", data_df[fea_name].map(self.ID[fea_name+"_mean"]))
            data_df[fea_name+"_mean"] = data_df[fea_name+"_mean"].fillna(data_df[fea_name+"_mean"].mean())
        self.add2list(len(index2name)-1, fea_name+"_mean", stage)
    
    
    def getCount(self, data_df, fea_name, stage):
        if stage == "train":
            self.ID[fea_name+"_count"] = dict(data_df[fea_name].value_counts())
            data_df.insert(len(data_df.columns)-1, fea_name+"_count", data_df[fea_name].map(self.ID[fea_name+"_count"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_count", data_df[fea_name].map(self.ID[fea_name+"_count"]))
            data_df[fea_name+"_count"] = data_df[fea_name+"_count"].fillna(data_df[fea_name+"_count"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_count", stage)
    
    
    def getMax(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_max"][list(set(v[fea_name]))[0]] = v["target"].max()
            data_df.insert(len(data_df.columns)-1, fea_name+"_max", data_df[fea_name].map(self.ID[fea_name+"_max"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_max", data_df[fea_name].map(self.ID[fea_name+"_max"]))
            data_df[fea_name+"_max"] = data_df[fea_name+"_max"].fillna(data_df[fea_name+"_max"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_max", stage)
    
    
    def getMin(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_min"][list(set(v[fea_name]))[0]] = v["target"].min()
            data_df.insert(len(data_df.columns)-1, fea_name+"_min", data_df[fea_name].map(self.ID[fea_name+"_min"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_min", data_df[fea_name].map(self.ID[fea_name+"_min"]))
            data_df[fea_name+"_min"] = data_df[fea_name+"_min"].fillna(data_df[fea_name+"_min"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_min", stage)
    
    
    def getMedian(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_median"][list(set(v[fea_name]))[0]] = v["target"].median()
            data_df.insert(len(data_df.columns)-1, fea_name+"_median", data_df[fea_name].map(self.ID[fea_name+"_median"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_median", data_df[fea_name].map(self.ID[fea_name+"_median"]))
            data_df[fea_name+"_median"] = data_df[fea_name+"_median"].fillna(data_df[fea_name+"_median"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_median", stage)
    
    
    def getSum(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_sum"][list(set(v[fea_name]))[0]] = v["target"].sum()
            data_df.insert(len(data_df.columns)-1, fea_name+"_sum", data_df[fea_name].map(self.ID[fea_name+"_sum"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_sum", data_df[fea_name].map(self.ID[fea_name+"_sum"]))
            data_df[fea_name+"_sum"] = data_df[fea_name+"_sum"].fillna(data_df[fea_name+"_sum"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_sum", stage)
    
    
    def getStd(self, data_df, fea_name, stage):
        if stage == "train":
            cs = data_df.loc[:, [fea_name, "target"]]
            cs = cs.groupby(fea_name)
            for k, v in cs:
                self.ID[fea_name+"_std"][list(set(v[fea_name]))[0]] = v["target"].std()
            data_df.insert(len(data_df.columns)-1, fea_name+"_std", data_df[fea_name].map(self.ID[fea_name+"_std"]))
        else:
            data_df.insert(len(data_df.columns)-1, fea_name+"_std", data_df[fea_name].map(self.ID[fea_name+"_std"]))
            data_df[fea_name+"_std"] = data_df[fea_name+"_std"].fillna(data_df[fea_name+"_std"].value_counts().index[0])
        self.add2list(len(index2name)-1, fea_name+"_std", stage)
        
    def newFeature(self, data_df, stage="train"):
        # 构造特征
        pd.set_option('mode.use_inf_as_na', True)

        data_df.insert(len(data_df.columns)-1, "降价比率", (data_df["新车价"] - data_df["target"]) / data_df["新车价"])
        self.add2list(len(index2name)-1, "降价比率", stage)
        
        data_df.insert(len(data_df.columns)-1,"展销是否为假期",data_df["展销时间"].map(lambda x: is_holiday(x)))
        self.add2list_cat(len(index2name)-1, "展销是否为假期", stage)
        
        data_df.insert(len(data_df.columns)-1,"是否转户", data_df["过户次数"].map(lambda x: int(x > 0)))
        # self.add2list(len(index2name)-1, "是否转户", stage)
        # self.add2list_cat(len(index2name)-1, "是否转户", stage)
        
        data_df.insert(len(data_df.columns)-1, "注册年份", data_df["注册日期"].map(lambda x: x.year))
        self.add2list(len(index2name)-1, "注册年份", stage)

        data_df.insert(len(data_df.columns)-1, "Feature_12_num_sqrt", data_df["Feature_12"].map(lambda x: eval(x) ** (1./3.)))
        self.add2list(len(index2name)-1, "Feature_12_num_sqrt", stage)

        data_df.insert(len(data_df.columns)-1, "Feature_12_0", data_df["Feature_12"].map(lambda x: int(x.split('*')[0])))
        self.add2list(len(index2name)-1, "Feature_12_0", stage)

        data_df.insert(len(data_df.columns)-1, "Feature_12_1", data_df["Feature_12"].map(lambda x: int(x.split('*')[1])))
        self.add2list(len(index2name)-1, "Feature_12_1", stage)

        data_df.insert(len(data_df.columns)-1, "Feature_12_2", data_df["Feature_12"].map(lambda x: int(x.split('*')[2])))
        self.add2list(len(index2name)-1, "Feature_12_2", stage)
        
        data_df.insert(len(data_df.columns)-1, "汽车年龄", (data_df["展销时间"] - data_df["上牌日期"]).map(lambda x: x.days))
        # self.add2list(len(index2name)-1, "汽车年龄", stage)
        
        data_df.insert(len(data_df.columns)-1, "降价次数", data_df["updatePriceTimeJson"].map(lambda x: len(x)-1))
        self.add2list(len(index2name)-1, "降价次数", stage)
        
        data_df.insert(len(data_df.columns)-1, "降价比例", data_df[["pushPrice", "updatePriceTimeJson"]].apply(lambda x: (x["pushPrice"] - float(list(x["updatePriceTimeJson"].values())[-1])) / x["pushPrice"], axis=1))
        self.add2list(len(index2name)-1, "降价比例", stage)


        self.getSum(data_df, "车辆所在城市 id", stage)
        
        self.getMean(data_df, "品牌 id", stage)
        self.getCount(data_df, "品牌 id", stage)
        self.getMax(data_df, "品牌 id", stage)
        self.getMin(data_df, "品牌 id", stage)
        self.getMedian(data_df, "品牌 id", stage)
        self.getSum(data_df, "品牌 id", stage)
        
        # self.getMean(data_df, "是否转户", stage)
        self.getMean(data_df, "厂商类型", stage)
        # self.getMean(data_df, "燃油类型", stage)
        self.getMean(data_df, "国标码", stage)
        self.getMean(data_df, "年款", stage)
        self.getMean(data_df, "变速箱", stage)
        self.getMean(data_df, "Feature_2", stage)
        # self.getMean(data_df, "Feature_6", stage)
        # self.getMean(data_df, "Feature_8", stage)
        # self.getMean(data_df, "Feature_9", stage)
        # self.getMean(data_df, "展销是否为假期", stage)
        # self.getCount(data_df, "车型 id", stage)
        
        data_df["新车价"] = np.log(data_df["新车价"])
        data_df.insert(len(data_df.columns)-1, "评估方法1", np.log(data_df.apply(lambda x: x["新车价"] * (0.15 + 0.85*(1 - 0.11*min(3, max(0., x["汽车年龄"] / 365)) - 0.1 * min(4., max(0, x["汽车年龄"] / 365-3)) - 0.09 * min(4., max(0, x["汽车年龄"] / 365-7)))), axis=1)))
        self.add2list(len(index2name)-1, "评估方法1", stage)
        data_df.insert(len(data_df.columns)-1, "评估方法2", np.log(data_df.apply(lambda x: x["新车价"] * (15 - 5 * min(6, max(0, x["里程"])) / 6 - 4 * min(6, max(0, x["里程"]-6)) / 6 - 3 * min(6, max(0, x["里程"]-12)) / 6 - 2 * min(6, max(0, x["里程"]-18)) / 6 - min(6, max(0, x["里程"]-24)) / 6) / 15, axis=1)))
        self.add2list(len(index2name)-1, "评估方法2", stage)
        data_df["评估方法2"].fillna(data_df["评估方法1"], inplace=True)

# 生成新特征
FE = FeatureEngine()
FE.newFeature(tr, "train")
FE.newFeature(te, "test")
# FE.crossFeature(tr, "train")
# FE.crossFeature(te, "test")

# 取出numerical特征和categories特征
tr_x = tr.iloc[:, :-1]
tr_y = tr[["sday"]]
# te_x = te.iloc[:, :-1]
# te_y = te[["sday"]]
tr_x_num = tr[numerical].astype("float")
te_x_num = te[numerical].astype("float")
tr_x_cat = tr[category]
te_x_cat = te[category]

# 特征归一化： (x-min) / (max-min)
for _ in tr_x_num:
    if not _ in ["厂商类型", "载客人数", "Feature_8", "是否转户"]:
        min_ = tr_x_num[_].min()
        max_ = tr_x_num[_].max()
        tr_x_num[_] = (tr_x_num[_] - min_) / (max_ - min_)
        te_x_num[_] = (te_x_num[_] - min_) / (max_ - min_)

tr_x_num_array = np.array(tr_x_num)
tr_y_array = np.array(tr_y)
te_x_num_array = np.array(te_x_num)

plt.figure(figsize=(20,10))
tr_num = pd.concat([tr_x_num, tr_y], axis=1)
sns.heatmap(tr_num.corr(), annot=True, fmt='.2f')
plt.savefig(root2 / "Q3_2.jpg")

from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from pprint import pprint

# 优化结果：模型参数(使用hebo进行参数优化，因为一次优化的时间消耗大，因此将优化结果存了下来)
lgbParams = {'learning_rate': 0.010035982594961306, 'subsample': 0.35028462644730735, 'colsample_bytree': 0.3796965007412236, 'reg_lambda': 0.0006865914755758266, 'reg_alpha': 0.0026163019185854106, 'num_leaves': 189, 'max_depth': 37, 'n_estimators': 1965, 'objective': 'mse', 'min_child_samples': 25}
xgbParams = {'learning_rate': 0.010211967985703159, 'subsample': 0.30001115463199896, 'colsample_bytree': 0.48460986059028555, 'lambda': 0.032780563044634864, 'alpha': 0.06981554843421037, 'max_depth': 26, 'min_child_weight': 10, 'n_estimators': 1845}
catParams = {'learning_rate': 0.08491056514750207, 'l2_leaf_reg': 1, 'max_depth': 8, 'n_estimators': 2000, 'min_child_samples': 25, 'loss_function': 'RMSE', 'verbose': False, 'task_type': 'CPU'}

from xgboost import plot_importance
from numpy import sort
from sklearn.feature_selection import SelectFromModel

# Fit model using each importance as a threshold
model_XGB = XGBRegressor(**xgbParams).fit(tr_x_num, tr_y)
thresholds = sort(model_XGB.feature_importances_)
f_i = dict(zip(numerical, model_XGB.feature_importances_))
f_i = sorted(f_i.items(), key=lambda x: x[1])
fig,ax = plt.subplots(figsize=(15,15))
plot_importance(model_XGB, importance_type="weight", ax=ax)
plt.savefig(root2 / "Q3_3.jpg")

for name, thresh in f_i:
  # select features using threshold
    selection = SelectFromModel(model_XGB, threshold=thresh, prefit=True)
    select_X_train = selection.transform(tr_x_num)
  # train model
    selection_model = XGBRegressor(**xgbParams)
    selection_model.fit(select_X_train, tr_y)
  # eval model
    select_X_test = selection.transform(te_x_num)
    y_pred = selection_model.predict(select_X_test)
    accuracy = metric(pt_tr.inverse_transform(y_pred.reshape(-1, 1)) + te[["fday"]], np.array(te[["成交周期"]]))
    print("Feature Name=%s Thresh=%.3f, n=%d, Accuracy: %.2f" % (name, thresh, select_X_train.shape[1], accuracy))

from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone
import numpy as np
 
# stacking
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
 
    # 将原来的模型clone出来，并且进行实现fit功能
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
 
        #对于每个模型，使用交叉验证的方法来训练初级学习器，并且得到次级训练集
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                self.base_models_[i].append(instance)
                out_of_fold_predictions[holdout_index, i] = y_pred
 
        # 使用次级训练集来训练次级学习器
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
 
    #在上面的fit方法当中，我们已经将我们训练出来的初级学习器和次级学习器保存下来了
    #predict的时候只需要用这些学习器构造我们的次级预测数据集并且进行预测就可以了
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

base_models = [
               CatBoostRegressor(**catParams),
               LGBMRegressor(**lgbParams),
               XGBRegressor(**xgbParams),
              ]

meta_model = LinearRegression()
stacking_model = StackingAveragedModels(base_models=base_models, meta_model=meta_model)

stacking_model.fit(tr_x_num_array, tr_y_array.ravel())

y_predict = stacking_model.predict(te_x_num_array)

res = pt_tr.inverse_transform(y_predict.reshape(-1, 1)) + te[["fday"]]

res.insert(0, "carid", te["carid"])

res["fday"] = res["fday"].astype("int")

res.to_csv(root / "Q3.txt", index=False, sep='\t', header=None)